# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress
from itertools import islice 
import math

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
print(cities)

['ushuaia', 'guerrero negro', 'menongue', 'hobart', 'rikitea', 'okhotsk', 'cherskiy', 'cabo san lucas', 'mahebourg', 'nizwa', 'salalah', 'berlevag', 'hilo', 'takoradi', 'georgetown', 'roald', 'trinidad', 'cape town', 'svetlaya', 'albany', 'goderich', 'khatanga', 'narsaq', 'saskylakh', 'jamestown', 'lorengau', 'grand river south east', 'qaanaaq', 'ambilobe', 'tuatapere', 'margate', 'svetlyy', 'kaitangata', 'ferkessedougou', 'atuona', 'lolua', 'talnakh', 'san patricio', 'carnarvon', 'zhongshu', 'hermanus', 'bredasdorp', 'longyearbyen', 'belushya guba', 'mwense', 'iqaluit', 'guelengdeng', 'nelson bay', 'thompson', 'attawapiskat', 'bluff', 'betare oya', 'bhimunipatnam', 'busselton', 'yellowknife', 'ancud', 'prainha', 'port stanley', 'zhicheng', 'norman wells', 'punta arenas', 'amderma', 'touros', 'codrington', 'severo-kurilsk', 'aykhal', 'port alfred', 'katsuura', 'salinopolis', 'petropavlovsk-kamchatskiy', 'saint-augustin', 'ardakan', 'weihe', 'visnes', 'new norfolk', 'ribeira grande', 'v

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# Create a dictionary to hold each city name as a key
# and a list of city weather data as values
all_cities_weather = {}

# Define function to create a dictionary containing weather data 
# for a single city, and append it to the dictionary.
def create_city_dict(city):
    # Create a list to hold weather data for a single city
    city_weather = []

    # Save config information
    url = "http://api.openweathermap.org/data/2.5/weather?"

    # Build query URL
    query_url = f"{url}appid={weather_api_key}&q={city}&units=Imperial"

    # Read JSON data
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()

    # Set variables to hold relevant data
    city_name = weather_json["name"]
    lat = weather_json["coord"]["lat"]
    lon = weather_json["coord"]["lon"]
    max_temp = weather_json["main"]["temp_max"]
    cloud = weather_json["clouds"]["all"]
    wind_speed = weather_json["wind"]["speed"]
    humidity = weather_json["main"]["humidity"]
    date = weather_json["dt"]
    id_code = weather_json["id"]
    country = weather_json["sys"]["country"]

    # Append variables to list
    city_weather.append(city_name)
    city_weather.append(cloud)
    city_weather.append(country)
    city_weather.append(date)
    city_weather.append(humidity)
    city_weather.append(lat)
    city_weather.append(lon)
    city_weather.append(max_temp)
    city_weather.append(wind_speed)

    # Update all_cities_weather dictionary to include data for that city
    all_cities_weather.update({city_name:city_weather})
    
    return city_weather

In [34]:
# Divide the cities list into groups

# Input list initialization
Inputt = iter(cities)

# List of leghts in which to split
length_to_split = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 26]

# Split using islice
city_split = [list(islice(Inputt, elem))
                 for elem in length_to_split]

# Perform a loop of API calls

try:
    for x in range(0, math.floor(len(cities)/50+1)):
        try:
            for y in range(0, 49):
                print(f"Processing Record {y+1} of Set {x+1} | {cities[x*50+y]}")
        except KeyError:
            
        # Delay 1.5 seconds between API calls
        time.sleep(1.5)
except IndexError:
    print(f"------------------------------") 
    print(f"Data Retrieval Complete") 
    print(f"------------------------------")
        
city = create_city_dict(cities[1])
print(city)
print(math.floor(len(cities)/50))

# try:
# except KeyError:
#     print("City not found. Skipping...")
        
# print(all_cities_weather)

Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | guerrero negro
Processing Record 3 of Set 1 | menongue
Processing Record 4 of Set 1 | hobart
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | okhotsk
Processing Record 7 of Set 1 | cherskiy
Processing Record 8 of Set 1 | cabo san lucas
Processing Record 9 of Set 1 | mahebourg
Processing Record 10 of Set 1 | nizwa
Processing Record 11 of Set 1 | salalah
Processing Record 12 of Set 1 | berlevag
Processing Record 13 of Set 1 | hilo
Processing Record 14 of Set 1 | takoradi
Processing Record 15 of Set 1 | georgetown
Processing Record 16 of Set 1 | roald
Processing Record 17 of Set 1 | trinidad
Processing Record 18 of Set 1 | cape town
Processing Record 19 of Set 1 | svetlaya
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 | goderich
Processing Record 22 of Set 1 | khatanga
Processing Record 23 of Set 1 | narsaq
Processing Record 24 of Set 1 | saskylakh
Processing Record 25 of Set 

In [ ]:
print(json.dumps(weather_json, indent=4))

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
Summary = pd.DataFrame.from_dict(all_cities_weather, orient="index")

Summary = Summary.rename(columns=
                        {0:"City",
                        1:"Cloudiness",
                        2:"Country",
                        3:"Date",
                        4:"Humidity",
                        5:"Lat",
                        6:"Lng",
                        7:"Max Temp",
                        8:"Wind Speed"}
                        )
Summary
Summary.reset_index(drop=True)

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots


In [ ]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression